In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.utils.data as data

from models import OmniwardModel
from data import RaceDataset

from racing_utils.utils import closest_point_idx, cyclic_slice, determine_direction_of_bound, rotate_into_map_coord
from racing_utils.torch_related import TensorStandardScaler, calc_reward_and_penalty, scale_batch_and_to_device

In [ ]:
# torch.backends.cudnn.determinstic = True
# torch.backends.cudnn.benchmark = False
torch.backends.cudnn.benchmark = True

In [ ]:
NUM_STEPS_AHEAD_TRAJ = 150
NUM_STEPS_AHEAD_ACT = 10
NUM_STEPS_AHEAD_BOUND = 50
NUM_STEPS_CENTERLINE = 300
CENTERLINE_DECIMATION = 1
PATH_TO_F1TENTH_GYM = Path('../f1tenth_gym')
FULL_STATE_FOR_TRAJECTORY = False

DATASET_SUFFIX = '_tiny' # '_tiny' or ''

DEVICE = 'cuda:0' # 'cuda:0'

BATCH_SIZE = 512

PROB_FLIP = 0.5

In [ ]:
unpickled = pd.read_pickle('./data/valid_tiny/2021-10-25_12_36_28.612196.pkl')
one_race = unpickled['data']
additional_data = unpickled['additional_data']

In [ ]:
centerline = additional_data['centerline'][::CENTERLINE_DECIMATION]
lookahead_distance = additional_data['lookahead_distance']
speed_setpoint = additional_data['speed_setpoint']
tire_force_max = additional_data['tire_force_max']

In [ ]:
one_race.head()

# We need to figure out if the bounds go in the same direction as the car is driving

In [ ]:
start_position, end_position = one_race.loc[[0, NUM_STEPS_AHEAD_TRAJ], 'position']

In [ ]:
bounds = []
bound_directions = []
for csv_file in ['interior.csv', 'exterior.csv']:
    bound = pd.read_csv(PATH_TO_F1TENTH_GYM / 'maps' / csv_file, header=None).values
    direction = determine_direction_of_bound(bound, start_position, end_position)
    bound_directions.append(direction)
    bound = bound[::direction]
    bounds.append(bound)

In [ ]:
row_id = 120 + len(one_race) // 4
row = one_race.iloc[row_id]

position = row['position']
yaw = row['yaw']

In [ ]:
closest_bound_indices = [closest_point_idx(position, bound) for bound in bounds] 

In [ ]:
bound_slices = [
    cyclic_slice(bound, closest_idx, NUM_STEPS_AHEAD_BOUND)
    for bound, closest_idx in zip(bounds, closest_bound_indices)
]

In [ ]:
local_positions = np.stack(one_race['position'].iloc[row_id:row_id+NUM_STEPS_AHEAD_TRAJ].values)

for bound_slice in bound_slices:
    plt.scatter(bound_slice[:, 0], bound_slice[:, 1])
    
plt.scatter(local_positions[:, 0], local_positions[:, 1])
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
local_positions = np.stack(one_race['position'].iloc[row_id:row_id+NUM_STEPS_AHEAD_TRAJ].values)

closest_centerline_idx = closest_point_idx(local_positions[0], centerline)
centerline_ahead = cyclic_slice(centerline, closest_centerline_idx, NUM_STEPS_CENTERLINE)
centerline_ahead = rotate_into_map_coord(centerline_ahead - position, -yaw)
plt.scatter(centerline_ahead[:, 0], centerline_ahead[:, 1], color='r', alpha=0.2)

for bound_slice in bound_slices:
    bound_slice = rotate_into_map_coord(bound_slice - position, -yaw)
    plt.scatter(bound_slice[:, 0], bound_slice[:, 1], color='gray', alpha=0.2)
        
local_positions = rotate_into_map_coord(local_positions - position, -yaw)
plt.scatter(local_positions[:, 0], local_positions[:, 1], alpha=0.1)
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
train_dataset = RaceDataset(
    NUM_STEPS_AHEAD_TRAJ,
    NUM_STEPS_AHEAD_ACT,
    NUM_STEPS_AHEAD_BOUND,
    NUM_STEPS_CENTERLINE,
    f'./data/train{DATASET_SUFFIX}',
    CENTERLINE_DECIMATION,
    prob_flip=PROB_FLIP,
    full_state_for_trajectory=FULL_STATE_FOR_TRAJECTORY,
)

In [ ]:
features, targets = train_dataset[1000]

In [ ]:
left_bound, right_bound = features['left_bound'], features['right_bound']
left_bound = left_bound.reshape(-1, 2)
right_bound = right_bound.reshape(-1, 2)
plt.scatter(left_bound[:, 0], left_bound[:, 1], color='gray', alpha=0.2)
plt.scatter(right_bound[:, 0], right_bound[:, 1], color='gray', alpha=0.2)

centerline = features['centerline'].reshape(-1, 2)
plt.scatter(centerline[:, 0], centerline[:, 1], color='r', alpha=0.2)

trajectory = targets['trajectory']
trajectory = trajectory.reshape(-1, 2)
plt.scatter(trajectory[:, 0], trajectory[:, 1], color='blue', alpha=0.1)
plt.gca().set_aspect('equal')
plt.show();

In [ ]:
train_loader = data.DataLoader(train_dataset, BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

# Initilize the scalers
features_scalers = {key: TensorStandardScaler(DEVICE) for key in features.keys()}
targets_scalers = {key: TensorStandardScaler(DEVICE) for key in targets.keys()}

# Do a partial fit of the scalers for both the input features, and the targets
for features_batch, targets_batch in train_loader:
    for feature_name in features.keys():
        features_scalers[feature_name].partial_fit(features_batch[feature_name])

    for target_name in targets.keys():
        targets_scalers[target_name].partial_fit(targets_batch[target_name])


for feature_name in features.keys():
        features_scalers[feature_name].tensorfy()

for target_name in targets.keys():
    targets_scalers[target_name].tensorfy()

In [ ]:
valid_dataset = RaceDataset(
    NUM_STEPS_AHEAD_TRAJ,
    NUM_STEPS_AHEAD_ACT,
    NUM_STEPS_AHEAD_BOUND,
    NUM_STEPS_CENTERLINE,
    f'./data/valid{DATASET_SUFFIX}',
    CENTERLINE_DECIMATION,
    prob_flip=PROB_FLIP,
    full_state_for_trajectory=FULL_STATE_FOR_TRAJECTORY,
)

In [ ]:
state_size = len(features['state'])
contr_params_size = len(features['contr_params'])
centerline_size = len(features['centerline'])
trajectory_size = len(targets['trajectory'])
actuators_size = len(targets['speeds_and_deltas'])

num_stacked = 3
centerline_encoder_sizes = num_stacked * [centerline_size // 4]
middle_sizes = num_stacked * [(centerline_size + trajectory_size + actuators_size) // 4]
output_sizes = [centerline_size // 2, (trajectory_size + actuators_size) // 4, (trajectory_size + actuators_size) // 2]


omniward_model = OmniwardModel(
    state_size,
    contr_params_size,
    centerline_size,
    centerline_encoder_sizes,
    middle_sizes,
    output_sizes,
    trajectory_size,
    actuators_size,
)

In [ ]:
omniward_model.to(DEVICE)

In [ ]:
train_loader = data.DataLoader(train_dataset, BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = data.DataLoader(valid_dataset, BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

In [ ]:
trajectory_gamma = 0.99999
actuators_gamma = 0.8

trajectory_gammas = (trajectory_gamma ** np.r_[np.arange(trajectory_size // 2), np.arange(trajectory_size // 2)])
trajectory_gammas = torch.from_numpy(trajectory_gammas[::-1].copy()).to(DEVICE)

actuators_gammas = actuators_gamma ** np.r_[np.arange(actuators_size // 2), np.arange(actuators_size // 2)]
actuators_gammas = torch.from_numpy(actuators_gammas).to(DEVICE)

In [ ]:
def traj_act_mse(*, traj_coeff, act_coeff, traj_gammas, act_gammas, traj, traj_pred, act, act_pred):
    return (
        traj_coeff * (traj_gammas * (traj - traj_pred) ** 2).mean(),
        act_coeff * (act_gammas * (act - act_pred) ** 2).mean(),
    )

In [ ]:
PENALTY_SIGMA = 0.15

trajectory_mse_coeff = 1.0
actuators_mse_coeff = 1.0

optimizers = (
    [5, torch.optim.Adam(omniward_model.parameters(), lr=1e-3)],
    [10, torch.optim.Adam(omniward_model.parameters(), lr=1e-4)],
    [2, torch.optim.Adam(omniward_model.parameters(), lr=1e-5)],
)
train_traj_mses_for_plot = []
train_act_mses_for_plot = []
train_reward_mses_for_plot = []
train_penalty_mses_for_plot = []

valid_traj_mses_for_plot = []
valid_act_mses_for_plot = []
valid_reward_mses_for_plot = []
valid_penalty_mses_for_plot = []

epoch = 0
for num_epochs_per_optimizer_round, optimizer in optimizers:
    print(f'Optimizer: {optimizer}')
    for _ in range(num_epochs_per_optimizer_round):
        
        ############
        # Training #
        ############
        train_traj_mse = 0
        train_act_mse = 0
        train_reward_mse = 0
        train_penalty_mse = 0
        omniward_model.train()
        for features_batch, targets_batch in train_loader:
            centerline = features_batch['centerline'].to(DEVICE)
            left_bound = features_batch['left_bound'].to(DEVICE)
            right_bound = features_batch['right_bound'].to(DEVICE)
            trajectory = targets_batch['trajectory'].to(DEVICE)

            reward, penalty = calc_reward_and_penalty(trajectory, centerline, left_bound, right_bound, penalty_sigma=PENALTY_SIGMA)
            
            features_batch, targets_batch = scale_batch_and_to_device(DEVICE, features_scalers, targets_scalers, features_batch, targets_batch)
            
            preds = omniward_model(**features_batch)
            trajectory_pred, actuators_pred = preds['trajectory_pred'], preds['actuators_pred']
            
            trajectory = targets_batch['trajectory'].squeeze(dim=1)
            actuators = targets_batch['speeds_and_deltas'].squeeze(dim=1)
            
            traj_mse, act_mse = traj_act_mse(
                traj_coeff=trajectory_mse_coeff,
                act_coeff=actuators_mse_coeff,
                traj_gammas=trajectory_gammas,
                act_gammas=actuators_gammas,
                traj=trajectory,
                traj_pred=trajectory_pred,
                act=actuators,
                act_pred=actuators_pred,
            )
                    
            optimizer.zero_grad()
            
            (traj_mse + act_mse).backward()
            
            optimizer.step()

            train_traj_mse += float(traj_mse)
            train_act_mse += float(act_mse)

            trajectory_pred = targets_scalers['trajectory'].inverse_transform(trajectory_pred)
            reward_pred, penalty_pred = calc_reward_and_penalty(trajectory_pred, centerline, left_bound, right_bound, penalty_sigma=PENALTY_SIGMA)
            train_reward_mse += float(((reward - reward_pred) ** 2).mean())
            train_penalty_mse += float(((penalty - penalty_pred) ** 2).mean())

        avg_train_traj_mse = train_traj_mse / len(train_loader)
        avg_train_act_mse = train_act_mse / len(train_loader)
        avg_train_reward_mse = train_reward_mse / len(train_loader)
        avg_train_penalty_mse = train_penalty_mse / len(train_loader)
        print(f'\n{epoch}: train_traj_MSE: {avg_train_traj_mse:.3f}, train_act_MSE: {avg_train_act_mse:.3f}, train_reward_MSE: {avg_train_reward_mse:.3f}, train_penalty_MSE: {avg_train_penalty_mse:.3f}')
        train_traj_mses_for_plot.append(avg_train_traj_mse)
        train_act_mses_for_plot.append(avg_train_act_mse)
        train_reward_mses_for_plot.append(avg_train_reward_mse)
        train_penalty_mses_for_plot.append(avg_train_penalty_mse)
        
            
        ##############
        # Validation #
        ##############
        valid_traj_mse = 0
        valid_act_mse = 0
        valid_reward_mse = 0
        valid_penalty_mse = 0
        omniward_model.eval()
        for features_batch, targets_batch in valid_loader:
            centerline = features_batch['centerline'].to(DEVICE)
            left_bound = features_batch['left_bound'].to(DEVICE)
            right_bound = features_batch['right_bound'].to(DEVICE)
            trajectory = targets_batch['trajectory'].to(DEVICE)

            reward, penalty = calc_reward_and_penalty(trajectory, centerline, left_bound, right_bound, penalty_sigma=PENALTY_SIGMA)

            features_batch, targets_batch = scale_batch_and_to_device(DEVICE, features_scalers, targets_scalers, features_batch, targets_batch)
            
            with torch.no_grad():
                preds = omniward_model(**features_batch)
                trajectory_pred, actuators_pred = preds['trajectory_pred'], preds['actuators_pred']

            trajectory = targets_batch['trajectory'].squeeze(dim=1)
            actuators = targets_batch['speeds_and_deltas'].squeeze(dim=1)
            
            traj_mse, act_mse = traj_act_mse(
                traj_coeff=trajectory_mse_coeff,
                act_coeff=actuators_mse_coeff,
                traj_gammas=trajectory_gammas,
                act_gammas=actuators_gammas,
                traj=trajectory,
                traj_pred=trajectory_pred,
                act=actuators,
                act_pred=actuators_pred,
            )
                    
            valid_traj_mse += float(traj_mse)
            valid_act_mse += float(act_mse)

            trajectory_pred = targets_scalers['trajectory'].inverse_transform(trajectory_pred)
            reward_pred, penalty_pred = calc_reward_and_penalty(trajectory_pred, centerline, left_bound, right_bound, penalty_sigma=PENALTY_SIGMA)
            valid_reward_mse += float(((reward - reward_pred) ** 2).mean())
            valid_penalty_mse += float(((penalty - penalty_pred) ** 2).mean())
            
        avg_valid_traj_mse = valid_traj_mse / len(valid_loader)
        avg_valid_act_mse = valid_act_mse / len(valid_loader)
        avg_valid_reward_mse = valid_reward_mse / len(valid_loader)
        avg_valid_penalty_mse = valid_penalty_mse / len(valid_loader)
        print(f'{epoch}: valid_traj_MSE: {avg_valid_traj_mse:.3f}, valid_act_MSE: {avg_valid_act_mse:.3f}, valid_reward_MSE: {avg_valid_reward_mse:.3f}, valid_penalty_MSE: {avg_valid_penalty_mse:.3f}')
        valid_traj_mses_for_plot.append(avg_valid_traj_mse)
        valid_act_mses_for_plot.append(avg_valid_act_mse)
        valid_reward_mses_for_plot.append(avg_valid_reward_mse)
        valid_penalty_mses_for_plot.append(avg_valid_penalty_mse)

        epoch += 1

    plt.scatter(penalty.detach().cpu().numpy(), penalty_pred.detach().cpu().numpy(), alpha=0.15)
    plt.show()
    plt.scatter(reward.detach().cpu().numpy(), reward_pred.detach().cpu().numpy(), alpha=0.15)
    plt.show()

In [ ]:
%%timeit
with torch.inference_mode():
    preds = omniward_model(**features_batch)
    dupa, dupa2 = preds['trajectory_pred'], preds['actuators_pred']
    dupa = targets_scalers['trajectory'].inverse_transform(dupa)
    dupa, dupa2 = calc_reward_and_penalty(dupa, centerline, left_bound, right_bound, penalty_sigma=PENALTY_SIGMA)
            

In [ ]:
plt.plot(train_traj_mses_for_plot, label='train_traj_mse')
plt.plot(valid_traj_mses_for_plot, label='valid_traj_mse')
plt.legend()
plt.show()

plt.plot(train_act_mses_for_plot, label='train_act_mse')
plt.plot(valid_act_mses_for_plot, label='valid_act_mse')
plt.legend()
plt.show()

plt.plot(train_reward_mses_for_plot, label='train_reward_mse')
plt.plot(valid_reward_mses_for_plot, label='valid_reward_mse')
plt.legend()
plt.show()

plt.plot(train_penalty_mses_for_plot, label='train_penalty_mse')
plt.plot(valid_penalty_mses_for_plot, label='valid_penalty_mse')
plt.legend()
plt.show();

In [ ]:
features_batch = {
    feature_name: features_scalers[feature_name].inverse_transform(features_batch[feature_name], copy=True)
    for feature_name in features_batch.keys()
}
targets_batch = {
    target_name: targets_scalers[target_name].inverse_transform(targets_batch[target_name], copy=True)
    for target_name in targets_batch.keys()
}

centerline = features_batch['centerline']
centerline = centerline.reshape(len(centerline), -1, 2).cpu().numpy()

traj_pred = trajectory_pred.reshape(len(trajectory_pred), -1, 2).cpu().numpy()

right_bound = features_batch['right_bound']
right_bound = right_bound.reshape(len(right_bound), -1, 2).cpu().numpy()

left_bound = features_batch['left_bound']
left_bound = left_bound.reshape(len(left_bound), -1, 2).cpu().numpy()

trajectory = targets_batch['trajectory']
traj = trajectory.reshape(len(trajectory), -1, 2).cpu().numpy()

In [ ]:
for i in range(10):    
    positions = traj[i]
    positions_pred = traj_pred[i]

    fig, axes = plt.subplots(1, 2, figsize=(20, 10))

    axes[0].plot(positions_pred[:, 0], positions_pred[:, 1], alpha=0.3, linewidth=5)
    axes[0].plot(positions[:, 0], positions[:, 1], alpha=0.3, linewidth=5)
    axes[0].plot(centerline[i, :, 0], centerline[i, :, 1], alpha=0.1, linewidth=5, color='red')
    axes[0].scatter(right_bound[i, :, 0], right_bound[i, :, 1], color='gray', alpha=0.2)
    axes[0].scatter(left_bound[i, :, 0], left_bound[i, :, 1], color='gray', alpha=0.2)
    axes[0].set_aspect('equal')

    plt.show()

In [ ]:
idx = 1020

In [ ]:
features, targets = valid_dataset[idx]

In [ ]:
left_bound, right_bound = features['left_bound'], features['right_bound']
left_bound = left_bound.reshape(-1, 2)
right_bound = right_bound.reshape(-1, 2)
plt.scatter(left_bound[:, 0], left_bound[:, 1], color='gray', alpha=0.2)
plt.scatter(right_bound[:, 0], right_bound[:, 1], color='gray', alpha=0.2)

centerline = features['centerline'].reshape(-1, 2)
plt.scatter(centerline[:, 0], centerline[:, 1], color='r', alpha=0.2)

trajectory = targets['trajectory']
trajectory = trajectory.reshape(-1, 2)
plt.scatter(trajectory[:, 0], trajectory[:, 1], color='blue', alpha=0.1)
plt.gca().set_aspect('equal')
plt.show();

In [ ]:
features_batch = {
    feature_name: features_scalers[feature_name].transform(features_batch[feature_name], copy=True)
    for feature_name in features_batch.keys()
}
targets_batch = {
    target_name: targets_scalers[target_name].transform(targets_batch[target_name], copy=True)
    for target_name in targets_batch.keys()
}

In [ ]:
eta = 0.1
features, targets = valid_dataset[idx]
num_steps = 4
penalty_sigma = 0.1


features = {
    key: features_scalers[key].transform(torch.tensor(value, device=DEVICE, dtype=torch.float))
    for key, value in features.items()
}
targets = {
    key: targets_scalers[key].transform(torch.tensor(value, device=DEVICE, dtype=torch.float))
    for key, value in targets.items()
}


contr_params = features['contr_params']
num_contr_params = len(contr_params)
step_directions = torch.cat(
        [torch.zeros((1, num_contr_params))]
        + [-step * torch.eye(num_contr_params) for step in reversed(range(1, num_steps+1))]
        + [ step * torch.eye(num_contr_params) for step in range(1, num_steps+1)],
    axis=0
).to(DEVICE)

batch_size = step_directions.shape[0]

contr_params = contr_params[None] + eta * step_directions

state = torch.tile(features['state'], (batch_size, 1))
centerline = torch.tile(features['centerline'], (batch_size, 1))
left_bound = torch.tile(features['left_bound'], (batch_size, 1))
right_bound = torch.tile(features['right_bound'], (batch_size, 1))

with torch.inference_mode():
    preds = omniward_model(state, contr_params, centerline, None, None)
    trajectory_pred, actuators_pred = preds['trajectory_pred'], preds['actuators_pred']

trajectory_pred = targets_scalers['trajectory'].inverse_transform(trajectory_pred)
reward_pred, penalty_pred = calc_reward_and_penalty(trajectory_pred, centerline, left_bound, right_bound, penalty_sigma=penalty_sigma)
            

traj = trajectory_pred.reshape(len(trajectory_pred), -1, 2).cpu().numpy()


In [ ]:
speeds_and_deltas = targets_scalers['speeds_and_deltas'].inverse_transform(actuators_pred)[0].cpu()
speeds_and_deltas_gt = targets_scalers['speeds_and_deltas'].inverse_transform(targets['speeds_and_deltas']).cpu()

half = len(speeds_and_deltas) // 2
plt.plot(speeds_and_deltas[:half])
plt.plot(speeds_and_deltas_gt[:half])
plt.show()
plt.plot(speeds_and_deltas[half:])
plt.plot(speeds_and_deltas_gt[half:]);

In [ ]:
num_steps_for_grad = 4



grad_contr_param = np.zeros(num_contr_params)
x = eta * np.arange(-num_steps_for_grad, num_steps_for_grad+1)
for contr_param_idx in range(num_contr_params):
    rewards = []
    penalties = []
    for pred_step in range(2 * num_steps_for_grad):
        pred_idx = 1 + pred_step * num_contr_params + contr_param_idx
        rewards.append(float(reward_pred[pred_idx].cpu()))
        penalties.append(float(penalty_pred[pred_idx].cpu()))
        if pred_step == num_steps_for_grad:
            rewards.append(float(reward_pred[0].cpu()))
            penalties.append(float(penalty_pred[0].cpu()))
        
        

    coeffs = np.polyfit(x, rewards, deg=1)
    plt.plot(x, coeffs[0] * x + coeffs[1])
    plt.plot(x, rewards)
    plt.show()

    coeffs = np.polyfit(x, penalties, deg=1)
    plt.plot(x, coeffs[0] * x + coeffs[1])
    plt.plot(x, penalties)
    plt.show()

    print('--------------------------------------------------------')

    grad_contr_param[contr_param_idx] = coeffs[0]


In [ ]:
for dupa_idx in range(batch_size):
    positions = traj[dupa_idx]
    plt.scatter(positions[:, 0], positions[:, 1], alpha=0.1)
# plt.gca().set_aspect('equal')


In [ ]:
from racing_utils import GradientDriver


centerline_direction = 1
left_bound_direction = valid_dataset.traj_data[0]['bound_directions'][0]
right_bound_direction = valid_dataset.traj_data[0]['bound_directions'][1]

grad_driver = GradientDriver(
    centerline=additional_data['centerline'][::centerline_direction],
    num_steps_centerline=NUM_STEPS_CENTERLINE,

    left_bound=valid_dataset.bounds[0][::left_bound_direction],
    right_bound=valid_dataset.bounds[1],
    num_steps_ahead_bound=NUM_STEPS_AHEAD_BOUND,

    # Controller-related
    init_contr_params=features['contr_params'],

    # Model-related
    omniward_model=omniward_model,
    features_scalers=features_scalers,
    targets_scalers=targets_scalers,

    device=DEVICE,
)

grad_driver.plan(
    ranges=None,
    yaw=row['yaw'],
    pos_x=row['position'][0],
    pos_y=row['position'][1],
    linear_vel_x=row['v_x'],
    linear_vel_y=row['v_y'],
    angular_vel_z=row['omega'],
)

In [ ]:
first_speeds = []
first_speeds_pred = []
first_deltas = []
first_deltas_pred = []

for act, act_pred in zip(actuators.cpu().numpy(), actuators_pred.detach().cpu().numpy()):
    act = targets_scalers['speeds_and_deltas'].inverse_transform([act])[0]
    act_pred = targets_scalers['speeds_and_deltas'].inverse_transform([act_pred])[0]
    half = len(act) // 2
    plt.plot(act[:half], alpha=0.5)
    plt.plot(act_pred[:half])
    first_speeds.append(act[0])
    first_speeds_pred.append(act_pred[0])
    first_deltas.append(act[half])
    first_deltas_pred.append(act_pred[half])
    plt.show()
    plt.plot(act[half:], alpha=0.5)
    plt.plot(act_pred[half:])
    plt.show()